# Importing the dataset into a dataframe

In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk

import seaborn as sns

from sklearn.metrics import confusion_matrix
import sklearn.metrics as metrics
import sklearn.model_selection as ms
import sklearn.preprocessing as preprocess
from sklearn.model_selection import train_test_split


In [22]:
# !pip install pyarabic
# !pip install langdetect
# !pip install nltk

# from langdetect import detect
# import pyarabic.araby as araby
# nltk.download("all")


In [23]:
dataset_folder_path="/home/youssef/AUC/Spring22/CSCE493002 - Machine Learning/project/datasets"
df = pd.read_csv(dataset_folder_path+'/freqEncoded.csv')
df.head()

,label,text,stemmed_text,normalized,clean_text,freq
0,1,ممتاز نوعا ما . النظافة والموقع والتجهيز والشا...,متز نوع ما . نظف وقع جهز شاطيء . طعم,متز نوع ما . نظف وقع جهز شاطيء . طعم,"['متز', 'نوع', 'ما', 'نظف', 'وقع', 'جهز', 'شاط...","{'متز': 1, 'نوع': 1, 'ما': 1, 'نظف': 1, 'وقع':..."
1,1,أحد أسباب نجاح الإمارات أن كل شخص في هذه الدول...,احد سبب نجح امر ان كل شخص في هذه دول عشق ترب ....,احد سبب نجح امر ان كل شخص في هذه دول عشق ترب ....,"['احد', 'سبب', 'نجح', 'امر', 'كل', 'شخص', 'هذه...","{'احد': 1, 'سبب': 1, 'نجح': 1, 'امر': 2, 'كل':..."
2,1,هادفة .. وقوية. تنقلك من صخب شوارع القاهرة الى...,هدف .. وقي . نقل من صخب شرع قهر الى هدء جبل شي...,هدف .. وقي . نقل من صخب شرع قهر الى هدء جبل شي...,"['هدف', 'نقل', 'صخب', 'شرع', 'قهر', 'هدء', 'جب...","{'هدف': 1, 'نقل': 1, 'صخب': 1, 'شرع': 1, 'قهر'..."
3,1,خلصنا .. مبدئيا اللي مستني ابهار زي الفيل الاز...,خلص .. بدئ الل مست بهر زي فيل زرق ميقراش حسن.....,خلص .. بدء الل مست بهر زي فيل زرق ميقراش حسن.....,"['خلص', 'بدء', 'الل', 'بهر', 'زي', 'فيل', 'زرق...","{'خلص': 3, 'بدء': 1, 'الل': 1, 'بهر': 1, 'زي':..."
4,1,ياسات جلوريا جزء لا يتجزأ من دبي . فندق متكامل...,ياس جلر جزء لا جزأ من دبي . ندق كامل خدم ريح ن...,ياس جلر جزء لا جزء من دبي . ندق كامل خدم ريح ن...,"['ياس', 'جزء', 'لا', 'جزء', 'دبي', 'ندق', 'كام...","{'ياس': 1, 'جزء': 2, 'لا': 2, 'دبي': 1, 'ندق':..."


### Drop unecessary columns and neutral label

In [24]:
df.drop(["text","stemmed_text","normalized","clean_text"],axis=1,inplace=True)

In [25]:
df_copy=df[(df['label']!= 0)]

In [26]:
df_copy.reset_index(inplace=True,drop=True)
df_copy

,label,freq
0,1,"{'متز': 1, 'نوع': 1, 'ما': 1, 'نظف': 1, 'وقع':..."
1,1,"{'احد': 1, 'سبب': 1, 'نجح': 1, 'امر': 2, 'كل':..."
2,1,"{'هدف': 1, 'نقل': 1, 'صخب': 1, 'شرع': 1, 'قهر'..."
3,1,"{'خلص': 3, 'بدء': 1, 'الل': 1, 'بهر': 1, 'زي':..."
4,1,"{'ياس': 1, 'جزء': 2, 'لا': 2, 'دبي': 1, 'ندق':..."
...,...,...
66661,-1,"{'عرفش': 1, 'ليه': 1, 'كنت': 1, 'كمل': 1, 'وهي..."
66662,-1,"{'لا': 3, 'سحق': 1, 'يكون': 1, 'كنق': 1, 'لنه'..."
66663,-1,"{'ضعف': 1, 'جدا': 1, 'ولم': 1, 'متع': 1, 'به':..."
66664,-1,"{'ملة': 1, 'جدا': 1, 'حمد': 1, 'حسن': 1, 'علو'..."


In [27]:
dfN=df_copy

# Unique Dictionary

In [28]:


def get_unique(df,colname):
  sentences_arr=df[colname].to_numpy() #convert df to np array
  words=' '.join(sentences_arr).split(' ') # join all strings into one string then splitting to get the words
  words=np.array(words) #convert to np array
  unique,counts = np.unique(words,return_counts=True)
  return unique, counts

def get_unique_dict(unique,counts):
  unique_dict=dict(zip(unique,counts))
  unique_dict=dict(sorted(unique_dict.items(), key=lambda item: item[1],reverse=True))
  return unique_dict
def remove_redundant_words(unique_dict):
  stop_words=['من','على','عن','في','فى','و','ان','هذا','او','كتب','...','.','','الى','فيه','انه','قبل','//','..','،',':',"؟",'/']
  for k, v in list(unique_dict.items()):
      if(unique_dict[k]<100 or k in stop_words):
        del unique_dict[k]

  return unique_dict



In [29]:
#df_copy= pd.read_csv('freqEncoded.csv')
#unique,counts=get_unique(df_copy,"normalized")
unique,counts=get_unique(df_copy,"freq")
unique_dict=get_unique_dict(unique,counts)
unique_dict=remove_redundant_words(unique_dict)

In [30]:
print("There are %d Unique words:"%len(unique_dict))
unique_dict

There are 2838 Unique words:


{'1,': 1724707,
 '2,': 254179,
 '3,': 66119,
 '1}': 65678,
 '4,': 24199,
 "'جدا':": 16799,
 "'لا':": 16265,
 "'ما':": 13117,
 "'كان':": 12325,
 "'كثر':": 11793,
 "'كل':": 11732,
 "'قرء':": 11569,
 "'وجد':": 11393,
 "'عمل':": 11385,
 "'لم':": 11313,
 "'وقع':": 10642,
 '5,': 10607,
 "'غير':": 10025,
 "'ندق':": 9294,
 "'بعد':": 9224,
 "'غرف':": 9145,
 "'جمل':": 8904,
 "'روي':": 8738,
 "'لكن':": 8616,
 "'مع':": 8561,
 "'حدث':": 8098,
 "'خدم':": 8077,
 "'نظف':": 7907,
 "'نفس':": 7907,
 "'علم':": 7804,
 "'جمع':": 7229,
 "'التي':": 7228,
 "'الل':": 7046,
 "'هو':": 6993,
 "'فكر':": 6918,
 "'شخص':": 6779,
 "'وحد':": 6686,
 "'قرب':": 6616,
 "'بعض':": 6601,
 "'عجب':": 6543,
 "'ولا':": 6436,
 "'كانت':": 6341,
 "'شكل':": 6282,
 "'عند':": 6199,
 "'كلم':": 6178,
 "'عرف':": 6123,
 "'اخر':": 5959,
 "'رءع':": 5934,
 "'حتى':": 5912,
 "'الا':": 5863,
 "'شيء':": 5851,
 "'قدم':": 5827,
 "'دخل':": 5779,
 "'بين':": 5671,
 "'هذه':": 5606,
 "'شعر':": 5586,
 '6,': 5534,
 "'سلب':": 5350,
 "'نظر':": 5258,
 "'الذي'

In [31]:
df_copy

,label,freq
0,1,"{'متز': 1, 'نوع': 1, 'ما': 1, 'نظف': 1, 'وقع':..."
1,1,"{'احد': 1, 'سبب': 1, 'نجح': 1, 'امر': 2, 'كل':..."
2,1,"{'هدف': 1, 'نقل': 1, 'صخب': 1, 'شرع': 1, 'قهر'..."
3,1,"{'خلص': 3, 'بدء': 1, 'الل': 1, 'بهر': 1, 'زي':..."
4,1,"{'ياس': 1, 'جزء': 2, 'لا': 2, 'دبي': 1, 'ندق':..."
...,...,...
66661,-1,"{'عرفش': 1, 'ليه': 1, 'كنت': 1, 'كمل': 1, 'وهي..."
66662,-1,"{'لا': 3, 'سحق': 1, 'يكون': 1, 'كنق': 1, 'لنه'..."
66663,-1,"{'ضعف': 1, 'جدا': 1, 'ولم': 1, 'متع': 1, 'به':..."
66664,-1,"{'ملة': 1, 'جدا': 1, 'حمد': 1, 'حسن': 1, 'علو'..."


# TF-IDF

In [32]:
import ast
#converting string dict to dict
df_copy['freq']=df_copy['freq'].apply(lambda x: ast.literal_eval(x))
print(type(df_copy.head()["freq"][0]))

<class 'dict'>


/home/youssef/.local/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [33]:
#joined words into a sentence again to use sklearn model

df_copy["text"]=df_copy["freq"].apply(lambda freq_dict: ' '.join(list(freq_dict.keys())) )
df_copy

,label,freq,text
0,1,"{'متز': 1, 'نوع': 1, 'ما': 1, 'نظف': 1, 'وقع':...",متز نوع ما نظف وقع جهز شاطيء طعم
1,1,"{'احد': 1, 'سبب': 1, 'نجح': 1, 'امر': 2, 'كل':...",احد سبب نجح امر كل شخص هذه دول عشق ترب نحن نحب...
2,1,"{'هدف': 1, 'نقل': 1, 'صخب': 1, 'شرع': 1, 'قهر'...",هدف نقل صخب شرع قهر هدء جبل شيش عرف حقق ما جرى...
3,1,"{'خلص': 3, 'بدء': 1, 'الل': 1, 'بهر': 1, 'زي':...",خلص بدء الل بهر زي فيل زرق حمد راد خطى رحل قرء...
4,1,"{'ياس': 1, 'جزء': 2, 'لا': 2, 'دبي': 1, 'ندق':...",ياس جزء لا دبي ندق كامل خدم ريح نفس وجد
...,...,...,...
66661,-1,"{'عرفش': 1, 'ليه': 1, 'كنت': 1, 'كمل': 1, 'وهي...",عرفش ليه كنت كمل وهي مش عجب حدث بطء ممل روي اط...
66662,-1,"{'لا': 3, 'سحق': 1, 'يكون': 1, 'كنق': 1, 'لنه'...",لا سحق يكون كنق لنه سيء شي وجد خدم فطر صبح ستي...
66663,-1,"{'ضعف': 1, 'جدا': 1, 'ولم': 1, 'متع': 1, 'به':...",ضعف جدا ولم متع به كل قصه سرد لحل شهد بدن فكر
66664,-1,"{'ملة': 1, 'جدا': 1, 'حمد': 1, 'حسن': 1, 'علو'...",ملة جدا حمد حسن علو فنن وصف عند دقق حد ثني قرء...


In [34]:
#Splitting data
x=df_copy['text']
y=np.expand_dims(df_copy['label'],axis=1)
X_train,X_test,Y_train, Y_test = train_test_split(x, y, test_size=0.3, random_state=1)
print("Train: " ,X_train.shape,Y_train.shape,"Test: ",(X_test.shape,Y_test.shape))

Train:  (46666,) (46666, 1) Test:  ((20000,), (20000, 1))


# TF-IDF

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer= TfidfVectorizer()
tf_x_train = vectorizer.fit_transform(X_train)
tf_x_test = vectorizer.transform(X_test)

In [36]:
print(tf_x_train[1000])

  (0, 3054)	0.399087852883007
  (0, 530)	0.34295341672047236
  (0, 1238)	0.3213888542967605
  (0, 3274)	0.27977113661028613
  (0, 3322)	0.3532507371589668
  (0, 2397)	0.3197743819705713
  (0, 3140)	0.23655315983481154
  (0, 1253)	0.19516641960193287
  (0, 1683)	0.3833121697914325
  (0, 3361)	0.17187081033107315
  (0, 2709)	0.15743661547652255
  (0, 2717)	0.1385857309095193


# Models Comparison

### Imports

In [37]:
from sklearn.tree import DecisionTreeClassifier 
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
#SVM
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
#NN
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import classification_report
import sklearn.model_selection as modsel
from sklearn.model_selection import GridSearchCV

In [38]:
scoring = ['precision_macro', 'recall_macro','f1_macro','accuracy'] #used for CV

In [39]:
def convert_scores_to_df(scores):
    df=pd.DataFrame(scores)
    df.drop(columns=['fit_time','score_time'],inplace=True)
    return df
def display_heatmap(y_test,pred):
    matrix=confusion_matrix(y_test,pred)
    sns.heatmap(matrix,annot=True)
def print_cv_scores(scores):
    scores_df=convert_scores_to_df(scores)
    print(scores_df)
    print("Means:-\n",scores_df.mean())

## Nerual Networks

In [63]:
class AdamOptimizer:
    def __init__(self,beta1,beta2,alpha,eps=10e-8):
#         self.params=params
#         self.n_iter=n_iter
        self.beta1=beta1
        self.beta2=beta2
        self.alpha=alpha
        self.eps=eps
        self.ms=[]
        self.vs=[]
        
    def reset_params(self,layers):
        self.ms=[ 
                  [np.zeros_like(layer.W), np.zeros_like(layer.b)] 
                  for layer in layers      
                 ]
        self.vs=self.ms.copy()
        
    def update(self,layers,N):
        for i in range(len(layers)):
            self.ms[i][0]= self.beta1*self.ms[i][0]*(1.0-self.beta1)*layers[i].dW
            self.ms[i][1]= self.beta1*self.ms[i][1]*(1.0-self.beta1)*layers[i].db
            
            self.vs[i][0]= self.beta1*self.ms[i][0]*(1.0-self.beta2)*np.square(layers[i].dW)
            self.vs[i][1]= self.beta1*self.ms[i][1]*(1.0-self.beta2)*np.square(layers[i].db)
            
            
            deltaW = (-1 * self.alpha * self.ms[i][0]) / (np.sqrt(self.vs[i][0] + self.eps))
            deltab = (-1 * self.alpha * self.ms[i][1]) / (np.sqrt(self.vs[i][1] + self.eps))

            layers[i].W +=  deltaW/N
            layers[i].b +=  deltab/N
        
    

In [108]:
class Layer:
    
    ### activations
    def _relu(self,z):
        return np.maximum(0,z)
    def _diff_relu(self,z):
        dZ=np.array(z,copy=True)
        dZ[dZ<=0]=0
        dZ[dZ>0]=1
        return dZ
    
    def _identity(self,z):
        return z
    
    def _identity_diff(self,z):
        return np.ones_like(z)
    
    def _sigmoid(self,z):
        return (1/(1+np.exp(-1*z)))

    def _diff_sigmoid(self,z):
        return self._sigmoid(z)*(1-self._sigmoid(z))
    
    def _softmax(self,z):
        expZ= np.exp(z-np.max(z))
        return expZ/expZ.sum(axis=0, keepdims=True)
    def _diff_softmax(self,z):
        pass

    
    ###########

    def __init__(self,n_input,n_output, activation="identity",name=None):
        self.n_output= n_output
        self.n_input= n_input
        self.name= name
        
        if activation == "identity":
            self.activation = self._identity
            self.diff_act= self._identity_diff
        
        elif activation == "sigmoid":
            self.activation = self._sigmoid
            self.diff_act= self._diff_sigmoid
            
        elif activation == "softmax":
            self.activation=self._softmax
            self.diff_act=self._diff_softmax
        elif activation =="relu":
            self.activation=self._relu
            self.diff_act=self._diff_relu
        
            
        
        
        self.W= np.random.randn(self.n_output,self.n_input)*np.sqrt(2/self.n_input)
        self.b= np.random.randn(self.n_output,1)*np.sqrt(2/self.n_input)

        self.dW= np.zeros_like(self.W)
        self.db= np.zeros_like(self.b)
        
        self.Z= None
        self.Ai = None
    def print_shapes(self):
        print("W: ",self.W.shape)
        print("b: ",self.b.shape)
#         print("W@Ai: ",self.W@Ai)
    
    def forward(self,Ai): #data dim 
#         print("FWD")
        print("W:",self.W.shape)
        print("Ai:",Ai.shape)
#         print(self.b.shape)
        z =  np.add((self.W @ Ai),self.b)
#         print(z.shape)
        A = self.activation(z)
        print("A:",A.shape)
        print("----------------")

        
        self.Z = z
        self.Ai = Ai
        return A
    
    
    def backward(self,inp):
        
#         print("input shape: ",end='')
#         print(inp.shape)
       
        act_diff = self.diff_act(self.Z)
#         print("act_diff shape: ",end='')
#         print(act_diff.shape)
        
        tmp = inp * act_diff
#         print("tmp shape: ",end='')
#         print(tmp.shape)
        
        bet = tmp @ self.Ai.T # vector of 1s
#         print("bet shape: ",end='')
#         print(bet.shape)
        
        
        e = np.ones((self.Ai.shape[1],1))
        db = tmp @ e
#         print("db shape: ",end='')
#         print(db.shape)
        self.dW = (self.dW + bet)
        # print("dw:",self.dW.shape,"\nlen:",len(self.dW))
        self.db = self.db + db
        
        
        return self.W.T @ tmp
    def print_weights(self):
        if(self.name):
            print("name: ",self.name, end=" ")
        print("dW: ",self.dW, "W: ",self.W)
    
    def zeroing_delta(self):
        self.dW= np.zeros_like(self.W)
        self.db= np.zeros_like(self.b)
#########################################################################################################
class NN:
    
    ########
    ## losses
    def _MSE(self,y,yhat):
        a=np.square(yhat-y)
        a=np.sum(a)
        b= 1/(2*y.shape[1])
        return a*b

    ## diff losses
    def _diff_MSE(self,y,yhat):
        return (yhat-y)
    
    def _binary_cross_entropy(self,y,yhat):
        pass
#         return -(y*np.log(yhat)+(1-y)*np.log(1-yhat)).mean()
    def _diff_binary_cross_entropy(self,y,yhat):
        loss=-((y*np.log(yhat)+(1-y)*np.log(1-yhat)).mean())
        return loss
    
    #########
    
    def __init__(self,lr,optimizer,loss="binary_cross"):
        self.layers = []
        self.alpha= lr
        self.optimizer=optimizer
        
        if(loss=="binary_cross"):
            self.loss=self._binary_cross_entropy
            self.loss_diff=self._diff_binary_cross_entropy
        elif loss=="MSE":
            self.loss=self._MSE
            self.loss_diff=self._diff_MSE
        
    
 
    
    def forward(self,x_train):
        a=x_train
#         print(a.shape)
        for layer in self.layers:
            a = layer.forward(a)
            print(a.shape)
        
        return a
    
    def backward(self,input):
        gd = input
        for layer in self.layers[::-1]:
            gd = layer.backward(gd)
            
    def add_layer(self,n_input,n_output, activation="identity",name=None):
        self.layers.append(Layer(n_input,n_output, activation=activation,name=name))
        
    def fit(self, x_train,y_train, epochs=5): #data dim is MxN .. M no of examples.. N no of dimension
        
        M = x_train.shape[0]
        
        
        x_train = x_train.T
        y_train = y_train.T
        
        # print(x_train.shape,self.layers[0].W.shape)
        # print(y_train.shape)
        
        for i in range(epochs):
            self.optimizer.reset_params(self.layers)
            print("Epoche {}/{}".format(i+1,epochs))
            y_hat= self.forward(x_train)
            
            dl_dyhat = self.loss_diff(y_train,y_hat)
            print("loss=",dl_dyhat)

            
            self.backward(dl_dyhat)
            
            # update using GD
#             for i in range(len(self.layers)):
#                 # layers[i].dW=layers[i].dW/N
#                 # layers[i].db=layers[i].db/N
#                 self.layers[i].W = self.layers[i].W - self.alpha * (self.layers[i].dW/M)
#                 self.layers[i].b = self.layers[i].b - self.alpha * (self.layers[i].db/M)
            self.optimizer.update(self.layers,M)
            self.print_weights()
            
            # zeroing deltas
            for layer in self.layers:
                layer.zeroing_delta()
            
        print("Finished....") 
            
            
        

    
    def predict(self,x_test): #data dim is NxD .. N no of examples.. D no of dimension
#         print(x_test.shape)
        y_hat= self.forward(x_test.T).T
        y_hat[y_hat>0.5]=1
        y_hat[y_hat<=0.5]=-1
#         print(y_hat.shape)
        return y_hat
                    
    def print_weights(self):
        for i in range(len(self.layers)):
            print("layer i= ",i,end=" ")
            self.layers[i].print_weights()
    def print_shapes(self):
        for layer in self.layers:
            layer.print_shapes()

In [109]:
print(tf_x_train.shape)
print(tf_x_test.shape)
Y_train.reshape(Y_train.shape[0],1)
print(Y_train.shape)
print(Y_test.shape)

(46666, 3593)
(20000, 3593)
(46666, 1)
(20000, 1)


In [110]:

adam= AdamOptimizer(beta1 = 0.8,beta2 = 0.999,alpha=0.001)
nn = NN(lr=0.01,loss="binary_cross",optimizer=adam)

nn.add_layer(tf_x_train.shape[1],64,activation="relu",name="l1")
nn.add_layer(64,32,activation = "relu",name="l2")
nn.add_layer(32,16,activation = "relu",name="l3")
nn.add_layer(16,8,activation = "relu",name="l4")
nn.add_layer(8,1,activation = "sigmoid",name="l5")



In [111]:
nn.fit(tf_x_train,Y_train,20)


Epoche 1/20
W: (64, 3593)
Ai: (3593, 46666)
A: (64, 46666)
----------------
(64, 46666)
W: (32, 64)
Ai: (64, 46666)
A: (32, 46666)
----------------
(32, 46666)
W: (16, 32)
Ai: (32, 46666)
A: (16, 46666)
----------------
(16, 46666)
W: (8, 16)
Ai: (16, 46666)
A: (8, 46666)
----------------
(8, 46666)
W: (1, 8)
Ai: (8, 46666)
A: (1, 46666)
----------------
(1, 46666)
loss= 0.38793436343507115
layer i=  0 

NameError: name 'layers' is not defined

In [105]:
y_pred=nn.predict(tf_x_test)

W: (64, 3593)
Ai: (3593, 20000)
A: (64, 20000)
----------------
(64, 20000)
W: (32, 64)
Ai: (64, 20000)
A: (32, 20000)
----------------
(32, 20000)
W: (16, 32)
Ai: (32, 20000)
A: (16, 20000)
----------------
(16, 20000)
W: (8, 16)
Ai: (16, 20000)
A: (8, 20000)
----------------
(8, 20000)
W: (1, 8)
Ai: (8, 20000)
A: (1, 20000)
----------------
(1, 20000)


In [106]:
print(min(y_pred))
max(y_pred)


[-1.]


array([-1.])

In [68]:

from sklearn.metrics import confusion_matrix
import sklearn.metrics as metrics
import sklearn.model_selection as ms
import sklearn.preprocessing as preprocess
from sklearn.model_selection import train_test_split

In [69]:
# classification_report(y_pred,Y_test)
Y_test

array([[ 1],
       [ 1],
       [ 1],
       ...,
       [ 1],
       [ 1],
       [-1]])

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import numpy as np
import pandas as pd
import math
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import os

tf_list= pd.read_csv('/first.txt')
array_tf_list = np.array(tf_list)
test_list=pd.read_csv('/second.txt')
array_test_list = np.array(test_list)
tlabels= pd.read_csv('/third.txt')
array_tlabels= np.array(tlabels)

label_test=pd.read_csv('/fourth.txt')
arr_lt= np.array(label_test)
array_tf_list=np.reshape(array_tf_list,[256,5999])
train_label_onehot =np.zeros((10,5999))
for i in range(6000-1):
  value=array_tlabels[i]
  for row in range (1,10,1):
    if (value==row):
      train_label_onehot[value,i]=1

def sigmoid(Z):
    A = 1/(1+np.exp(-Z))
    cache = Z
    return A, cache

def relu(Z):
    A = np.maximum(0,Z)
    cache = Z
    return A, cache

def softmax(Z):
    e_x = np.exp(Z)
    A= e_x / np.sum(np.exp(Z))
    cache=Z
    return A,cache

def relu_backward(dA, cache):
    Z = cache
    dZ = np.array(dA, copy=True)
    dZ[Z <= 0] = 0
    assert (dZ.shape == Z.shape)
    return dZ

def sigmoid_backward(dA, cache):
    Z = cache
    s = 1/(1+np.exp(-Z))
    dZ = dA * s * (1-s)
    assert (dZ.shape == Z.shape)
    return dZ

def softmax_backward(Z,cache):
    Z=cache
    length=10
    dZ=np.zeros((6000-1,10))
    Z=np.transpose(Z)
    for row in range (0,5999):
            den=(np.sum(np.exp(Z[row,:])))*(np.sum(np.exp(Z[row,:])))
            for col in range (0,10):
                sums=0
                for j in range (0,10):
                    if (j!=col):
                        sums=sums+(math.exp(Z[row,j]))

                dZ[row,col]=(math.exp(Z[row,col])*sums)/den
    dZ=np.transpose(dZ)
    Z=np.transpose(Z)
    assert (dZ.shape == Z.shape)
    return dZ

def initialize_parameters_deep(mylay_dimensions):
    parameters = {}
    L = len(mylay_dimensions)
    for l in range(1, L):
        parameters['W' + str(l)] = np.random.randn(mylay_dimensions[l], mylay_dimensions[l-1]) / np.sqrt(mylay_dimensions[l-1]) #*0.01
        parameters['b' + str(l)] = np.zeros((mylay_dimensions[l], 1))
    return parameters

def for_front(A, W, b):
    Z = np.dot(W,A) +b
    cache = (A, W, b)
    assert(Z.shape == (W.shape[0], A.shape[1]))
    return Z, cache

def for_active(A_prev, W, b, activation):
    if activation == "sigmoid":
        Z, linear_cache = for_front(A_prev, W, b)
        A, activation_cache = sigmoid(Z)
    elif activation == "relu":
        Z, linear_cache = for_front(A_prev, W, b)
        A, activation_cache = relu(Z)
    elif activation == "softmax":
        Z, linear_cache = for_front(A_prev, W, b)
        A, activation_cache = softmax(Z)
    cache = (linear_cache, activation_cache)
    return A, cache

def my_mod_for(X, parameters):
    caches = []
    A = X
    L = len(parameters)
    for l in range(1, L):
        A_prev = A
        A, cache = for_active(A_prev, parameters['W' + str(l)], parameters['b' + str(l)], activation = "relu")
        caches.append(cache)
    AL, cache = for_active(A, parameters['W' + str(L)], parameters['b' + str(L)], activation = "softmax")
    caches.append(cache)
    return AL, caches

def calc_cost(AL, Y):
    m = Y.shape[1]
    cost = (-1 / m) * np.sum(np.multiply(Y, np.log(AL)) + np.multiply(1 - Y, np.log(1 - AL)))
    return cost

def linear_backward(dZ, cache):
    A_prev, W, b = cache
    m = A_prev.shape[1]
    dW = 1./m * np.dot(dZ,A_prev.T)
    db = (1/m)*np.sum(dZ, axis=1, keepdims=True);
    dA_prev = np.dot(W.T,dZ)
    return dA_prev, dW, db

def lin_back(dA, cache, activation):
    linear_cache, activation_cache = cache
    if activation == "relu":
        dZ = relu_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
    elif activation == "sigmoid":
        dZ = sigmoid_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
    elif activation == "softmax":
        dZ = softmax_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
    return dA_prev, dW, db

def my_mod_back(AL, Y, caches):
    arbit_arr = {}
    L = len(caches)
    dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))
    M=len(my_network_dimensions)
    current_cache = caches[M-2]
    arbit_arr["dA"+str(M-1)], arbit_arr["dW"+str(M-1)], arbit_arr["db"+str(M-1)] = lin_back(dAL, current_cache, activation = "softmax")#M-1
    for l in reversed(range(L-1)):
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = lin_back(arbit_arr["dA" + str(l + 2)], current_cache, activation = "relu")
        arbit_arr["dA" + str(l + 1)] = dA_prev_temp
        arbit_arr["dW" + str(l + 1)] = dW_temp
        arbit_arr["db" + str(l + 1)] = db_temp
    return arbit_arr

def change_weights(parameters, arbit_arr, learning_rate):
    for l in range(len_update-1):
        parameters["W" + str(l+1)] =parameters["W" + str(l+1)] - (learning_rate*arbit_arr["dW" + str(l+1)])
        parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - (learning_rate*arbit_arr["db" + str(l+1)])
    return parameters

def plot_graph(cost_plot):
    x_value=list(range(1,len(cost_plot)+1))
    plt.xlabel('iteration')
    plt.ylabel('cost')
    plt.plot(x_value,cost_plot,0.,color='g')
my_network_dimensions = [256,150,100,50,10]
len_update=len(my_network_dimensions)
def my_lay_mod(X, Y, my_network_dimensions, learning_rate , num_iterations , print_cost=False):#lr was 0.009
    costs = []
    cost_plot=np.zeros(num_iterations)
    parameters = initialize_parameters_deep(my_network_dimensions)
    for i in range(0, num_iterations):
        AL, caches = my_mod_for(X, parameters)
        cost =calc_cost(AL, Y)
        arbit_arr = my_mod_back(AL, Y, caches)
        parameters = change_weights(parameters, arbit_arr, learning_rate)
        cost_plot[i]=cost;
    plot_graph(cost_plot)
    return parameters
parameters = my_lay_mod(array_tf_list, train_label_onehot, my_network_dimensions,learning_rate = 0.005, num_iterations =50 , print_cost = True)
for weight in parameters:
    print(weight)